여러 종류의 classifiers 의 사용법을 익히기 위하여 앞서 다뤘던 영화 평점 분류 데이터를 이용합니다. Support Vector Machine 계열들은 데이터를 그대로 외우기 때문에 데이터의 크기가 커지면 모델의 크기도 커집니다. 그렇기 때문에 이번 실습에서는 데이터의 개수가 10k 인 작은 데이터를 이용합니다. 긍정 (label = 1), 부정 (label = -1) 의 리뷰 5k 씩으로 구성되어 있는 balanced dataset 입니다.

`dataset_dir` 폴더에는 `lovit_textmining_dataset` 패키지가 있고, 그 아래 하위 패키지로 `navermovie_comments` 가 존재합니다. `navermovie_comments` 의 위치를 sys.path 에 추가하면 간략하게 데이터를 로딩할 수도 있습니다.

In [1]:
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')

from config import dataset_dir
sys.path.append('{}/lovit_textmining_dataset/'.format(dataset_dir))

from navermovie_comments import load_sentiment_dataset

texts, x, y, idx_to_vocab = load_sentiment_dataset(data_name='10k', tokenize='komoran')
print(x.shape)
print(y.shape)

Dataset version
[navermovie_comments.data] is latest (0.0.1)
[navermovie_comments.models] is latest (0.0.1)
[navernews_10days.data] is latest (0.0.1)
[navernews_10days.models] is latest (0.0.1)
(10000, 4808)
(10000,)


## normalize

학습용 데이터 x는 row normalize를 미리 해두겠습니다. 기본은 norm='l2' 입니다. 벡터를 확률의 형식으로 (합이 1 이 되도록) 만들고 싶다면 norm 을 'l1' 으로 변경하면 됩니다.

```
x_norm_l1 = normalize(x, norm='l1')
```

In [2]:
from sklearn.preprocessing import normalize

x_norm = normalize(x, norm='l2')

## Logistic Regression

모델의 사용법을 알기 위함이니 학습과 검증 데이터를 따로 분류하지는 않겠습니다.

In [3]:
%time
from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression()
logistic_regression.fit(x_norm, y)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.7 µs


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

테스트는 여러 classifier 를 이용하여 반복할테니 함수로 만들어 둡니다. idxs 입력받아 이들의 true label 과 predicted label 을 확인합니다.

test samples 는 부정 리뷰 10 개와 긍정 리뷰 10 개 입니다. 모든 리뷰를 제대로 분류할 수 있는 것은 아닙니다. Unigram 의 한계이거나, 모델 구조의 한계일 수 있습니다. 또한 10k 개의 데이터셋 사이즈의 문제일 수도 있습니다. 각 모델별로 이와 같은 결과가 나오는 이유에 대하여 이해하는 것이 중요합니다.

Logistic regression 은 특정 단어가 존재하느냐에 영향을 가장 많이 받습니다. 영화 데이터에서는 10/SN 이 `10점준다` 라는 맥락에서 자주 이용되는 단어입니다.

In [4]:
def test(classifier, x, y, texts, idxs):
    predicts = classifier.predict(x[idxs])
    texts_ = [texts[idx] for idx in idxs]
    trues = y[idxs]    
    for text, true, pred in zip(texts_, trues, predicts):
        text_strf = ' '.join(text)
        true_strf = 'pos' if true == 1 else 'neg'
        pred_strf = 'pos' if pred == 1 else 'neg'
        print('[{} -> {}] {}'.format(true_strf, pred_strf, text_strf))

test_samples = np.asarray(list(range(0,10)) + list(range(5000, 5010)))
test(logistic_regression, x, y, texts, test_samples)

[neg -> pos] 재미없/VA 다/EC 이상/NNG 10/SN 자/NNB
[neg -> pos] 다크/NNP 나이트/NNP 라/NNG 이즈/NNP 에/JKB 이/NP 은/JX 놀라/NNP ㄴ/JX 의/JKG 거품/NNG 이/JKS 확실히/MAG 드러나/VV ㄴ/ETM 영화/NNG 시각/NNG 효과/NNG 빼/VV 곤/EC 아무것/NNG 도/JX 보/VV ㄹ게/EC 없/VA 는/ETM 망/NNG 작/NNG
[neg -> neg] 시사회/NNP 보고/JKB 꿀/NNG 잠/NNG 불면증/NNP 치료/NNP 에/JKB
[neg -> neg] 알/VV 고/EC 보/VX 니/EC 지구/NNG 이/VCP 라는/ETM 설정/NNG 은/JX 혹성탈출/NNP 하/VV 고/EC 똑같/VA 지/EC 않/VX 어/EC 실망/NNP
[neg -> pos] 놀라/VV ㄴ/ETM 각본/NNP 은/JX 천재/NNP 이자/NNP 놀라/NNP ㄴ/JX 의/JKG 동생/NNG 조나단/NNP 놀라/VV ㄴ/ETM 영원/NNG 하/XSV ㄴ/ETM 거장/NNG 스티븐 스필버그/NNP 감독/NNG 제작/NNG 참여/NNG 한스/NNP 짐머/NNP 까지/JX OST/SL 하/XSV 아/EC 주/VX ㄴ다면/EC 길/VA ㄴ/ETM 말/NNG 필요/NNG 없이/MAG 일단/MAG 서/VV ㄴ/ETM 감상/NNP 천재/NNG 성/XSN 이/JKS 다분/XR 하/XSA ㄴ/ETM 놀라/VV ㄴ/ETM 형제/NNP 는/JX 지금껏/MAG 실망/NNG 시키/XSV ㄴ/ETM 적/XSN 이/JKS 없/VA 기/ETN 에/JKB 그만큼/MAG 사람/NNG 들/XSN 이/JKS 믿/VV 고/EC 보/VX 는/ETM 것/NNB 같/VA 은/ETM 소리/NNP 하네/NNP 핵/NNG 노/NNP 재/XPN 므/NNP
[neg -> neg] ㅅㅂ/NA 이런/MM 영화/NNG 제일/NNG 싫/VA 은데/EC 몰입/NNP 도도/NNP 없/VA 고/EC 흥행/NNG 도/JX 패션왕/NNP 한테/JKB OO/SL 영화/NNG

Logistic Regression의 coefficient를 살펴보는 것은 이전에 다뤘습니다. 긍정, 부정을 의미하는 단어들을 살펴볼 수 있습니다.

In [5]:
# Coefficients를 array로 만듭니다. 
coefficients = logistic_regression.coef_.reshape(-1)

# Coefficient를 (index, coef)로 만든 뒤, coefficient 의 크기로 내림차순 정렬합니다. 
sorted_coefficients = sorted(enumerate(coefficients), key=lambda x:-x[1])

# 긍정적인 리뷰에 자주 등장하는 단어들을 다시 살펴봅니다.
for j, coef in sorted_coefficients[:10]:
    print('{} : {:.3}'.format(idx_to_vocab[j], coef))

관람객/NNG : 7.07
놀라/VV : 4.77
최고/NNG : 4.43
10/SN : 3.81
인생/NNP : 3.46
필요/NNG : 3.3
롭/XSA : 3.04
재미있/VA : 3.02
우주/NNG : 2.96
인터스텔라/NNP : 2.87


## Multi Layer Perceptron Classifier

Feedforward neural network 의 sklearn 의 이름이며, NN 은 parameter 를 해석하기가 조금 복잡합니다. 하지만 학습 방법은 Logistic Regression 과 동일합니다. 

Neural network 의 경우, hidden laber 의 층 수와, 각 층의 노드 수를 정해줄 수 있습니다. 아래 예시처럼 hidden layer 를 정하면, 2 개의 layers 에 각각 20, 5 개의 nodes 를 이용한다는 의미입니다. 

In [6]:
%%time
from sklearn.neural_network import MLPClassifier

neural_network = MLPClassifier(hidden_layer_sizes=(20,5))
neural_network.fit(x_norm, y)

test(neural_network, x, y, texts, test_samples)

[neg -> neg] 재미없/VA 다/EC 이상/NNG 10/SN 자/NNB
[neg -> neg] 다크/NNP 나이트/NNP 라/NNG 이즈/NNP 에/JKB 이/NP 은/JX 놀라/NNP ㄴ/JX 의/JKG 거품/NNG 이/JKS 확실히/MAG 드러나/VV ㄴ/ETM 영화/NNG 시각/NNG 효과/NNG 빼/VV 곤/EC 아무것/NNG 도/JX 보/VV ㄹ게/EC 없/VA 는/ETM 망/NNG 작/NNG
[neg -> neg] 시사회/NNP 보고/JKB 꿀/NNG 잠/NNG 불면증/NNP 치료/NNP 에/JKB
[neg -> neg] 알/VV 고/EC 보/VX 니/EC 지구/NNG 이/VCP 라는/ETM 설정/NNG 은/JX 혹성탈출/NNP 하/VV 고/EC 똑같/VA 지/EC 않/VX 어/EC 실망/NNP
[neg -> neg] 놀라/VV ㄴ/ETM 각본/NNP 은/JX 천재/NNP 이자/NNP 놀라/NNP ㄴ/JX 의/JKG 동생/NNG 조나단/NNP 놀라/VV ㄴ/ETM 영원/NNG 하/XSV ㄴ/ETM 거장/NNG 스티븐 스필버그/NNP 감독/NNG 제작/NNG 참여/NNG 한스/NNP 짐머/NNP 까지/JX OST/SL 하/XSV 아/EC 주/VX ㄴ다면/EC 길/VA ㄴ/ETM 말/NNG 필요/NNG 없이/MAG 일단/MAG 서/VV ㄴ/ETM 감상/NNP 천재/NNG 성/XSN 이/JKS 다분/XR 하/XSA ㄴ/ETM 놀라/VV ㄴ/ETM 형제/NNP 는/JX 지금껏/MAG 실망/NNG 시키/XSV ㄴ/ETM 적/XSN 이/JKS 없/VA 기/ETN 에/JKB 그만큼/MAG 사람/NNG 들/XSN 이/JKS 믿/VV 고/EC 보/VX 는/ETM 것/NNB 같/VA 은/ETM 소리/NNP 하네/NNP 핵/NNG 노/NNP 재/XPN 므/NNP
[neg -> neg] ㅅㅂ/NA 이런/MM 영화/NNG 제일/NNG 싫/VA 은데/EC 몰입/NNP 도도/NNP 없/VA 고/EC 흥행/NNG 도/JX 패션왕/NNP 한테/JKB OO/SL 영화/NNG

Coefficients 는 coefs_ 에 list 형식으로 저장됩니다. 

In [7]:
print(type(neural_network.coefs_))
print(len(neural_network.coefs_))

<class 'list'>
3


In [8]:
for i, coef_ in enumerate(neural_network.coefs_):
    print('layer {} : {}'.format(i+1, coef_.shape))

layer 1 : (4808, 20)
layer 2 : (20, 5)
layer 3 : (5, 1)


Intercept 는 $h_{t-1} \times \beta_t + intercept_t$ 처럼 이전 layer 의 output 값과 $\beta$ 의 내적에 더해지는 값입니다. Logistic regression 에서의 평행이동과 같습니다.

In [9]:
neural_network.intercepts_

[array([0.0815515 , 0.05732039, 0.43561834, 0.12969709, 0.12500567,
        0.12931301, 0.2722803 , 0.08082177, 0.4050362 , 0.06004531,
        0.07852185, 0.10111362, 0.19697818, 0.10313602, 0.06761073,
        0.09362888, 0.18652071, 0.31463787, 0.50370473, 0.09673633]),
 array([0.34643458, 0.06569037, 0.00279967, 0.376795  , 0.29684621]),
 array([0.31228303])]

## Support Vector Machine (Classifier)

SVM은 kernel을 정할 수 있습니다. default는 linear입니다. 

Linear kernel이 속도도 훨씬 빠릅니다. 

In [10]:
%%time
from sklearn.svm import SVC

rbf_SVM = SVC(C=1.0, kernel='rbf')
rbf_SVM.fit(x_norm, y)
test(rbf_SVM, x, y, texts, test_samples)

[neg -> neg] 재미없/VA 다/EC 이상/NNG 10/SN 자/NNB
[neg -> pos] 다크/NNP 나이트/NNP 라/NNG 이즈/NNP 에/JKB 이/NP 은/JX 놀라/NNP ㄴ/JX 의/JKG 거품/NNG 이/JKS 확실히/MAG 드러나/VV ㄴ/ETM 영화/NNG 시각/NNG 효과/NNG 빼/VV 곤/EC 아무것/NNG 도/JX 보/VV ㄹ게/EC 없/VA 는/ETM 망/NNG 작/NNG
[neg -> neg] 시사회/NNP 보고/JKB 꿀/NNG 잠/NNG 불면증/NNP 치료/NNP 에/JKB
[neg -> neg] 알/VV 고/EC 보/VX 니/EC 지구/NNG 이/VCP 라는/ETM 설정/NNG 은/JX 혹성탈출/NNP 하/VV 고/EC 똑같/VA 지/EC 않/VX 어/EC 실망/NNP
[neg -> pos] 놀라/VV ㄴ/ETM 각본/NNP 은/JX 천재/NNP 이자/NNP 놀라/NNP ㄴ/JX 의/JKG 동생/NNG 조나단/NNP 놀라/VV ㄴ/ETM 영원/NNG 하/XSV ㄴ/ETM 거장/NNG 스티븐 스필버그/NNP 감독/NNG 제작/NNG 참여/NNG 한스/NNP 짐머/NNP 까지/JX OST/SL 하/XSV 아/EC 주/VX ㄴ다면/EC 길/VA ㄴ/ETM 말/NNG 필요/NNG 없이/MAG 일단/MAG 서/VV ㄴ/ETM 감상/NNP 천재/NNG 성/XSN 이/JKS 다분/XR 하/XSA ㄴ/ETM 놀라/VV ㄴ/ETM 형제/NNP 는/JX 지금껏/MAG 실망/NNG 시키/XSV ㄴ/ETM 적/XSN 이/JKS 없/VA 기/ETN 에/JKB 그만큼/MAG 사람/NNG 들/XSN 이/JKS 믿/VV 고/EC 보/VX 는/ETM 것/NNB 같/VA 은/ETM 소리/NNP 하네/NNP 핵/NNG 노/NNP 재/XPN 므/NNP
[neg -> neg] ㅅㅂ/NA 이런/MM 영화/NNG 제일/NNG 싫/VA 은데/EC 몰입/NNP 도도/NNP 없/VA 고/EC 흥행/NNG 도/JX 패션왕/NNP 한테/JKB OO/SL 영화/NNG

In [11]:
type(rbf_SVM.support_vectors_)

scipy.sparse.csr.csr_matrix

이런 경우는 저도 자주 보지 못합니다만, 데이터를 모두 외워버렸습니다. k-nearest neighbor 와 다를 게 없습니다.

In [12]:
rbf_SVM.support_vectors_.shape

(10000, 4808)

In [13]:
%%time

linear_SVM = SVC(C=1.0, kernel='linear')
linear_SVM.fit(x_norm, y)
test(linear_SVM, x, y, texts, test_samples)

[neg -> pos] 재미없/VA 다/EC 이상/NNG 10/SN 자/NNB
[neg -> neg] 다크/NNP 나이트/NNP 라/NNG 이즈/NNP 에/JKB 이/NP 은/JX 놀라/NNP ㄴ/JX 의/JKG 거품/NNG 이/JKS 확실히/MAG 드러나/VV ㄴ/ETM 영화/NNG 시각/NNG 효과/NNG 빼/VV 곤/EC 아무것/NNG 도/JX 보/VV ㄹ게/EC 없/VA 는/ETM 망/NNG 작/NNG
[neg -> neg] 시사회/NNP 보고/JKB 꿀/NNG 잠/NNG 불면증/NNP 치료/NNP 에/JKB
[neg -> neg] 알/VV 고/EC 보/VX 니/EC 지구/NNG 이/VCP 라는/ETM 설정/NNG 은/JX 혹성탈출/NNP 하/VV 고/EC 똑같/VA 지/EC 않/VX 어/EC 실망/NNP
[neg -> pos] 놀라/VV ㄴ/ETM 각본/NNP 은/JX 천재/NNP 이자/NNP 놀라/NNP ㄴ/JX 의/JKG 동생/NNG 조나단/NNP 놀라/VV ㄴ/ETM 영원/NNG 하/XSV ㄴ/ETM 거장/NNG 스티븐 스필버그/NNP 감독/NNG 제작/NNG 참여/NNG 한스/NNP 짐머/NNP 까지/JX OST/SL 하/XSV 아/EC 주/VX ㄴ다면/EC 길/VA ㄴ/ETM 말/NNG 필요/NNG 없이/MAG 일단/MAG 서/VV ㄴ/ETM 감상/NNP 천재/NNG 성/XSN 이/JKS 다분/XR 하/XSA ㄴ/ETM 놀라/VV ㄴ/ETM 형제/NNP 는/JX 지금껏/MAG 실망/NNG 시키/XSV ㄴ/ETM 적/XSN 이/JKS 없/VA 기/ETN 에/JKB 그만큼/MAG 사람/NNG 들/XSN 이/JKS 믿/VV 고/EC 보/VX 는/ETM 것/NNB 같/VA 은/ETM 소리/NNP 하네/NNP 핵/NNG 노/NNP 재/XPN 므/NNP
[neg -> neg] ㅅㅂ/NA 이런/MM 영화/NNG 제일/NNG 싫/VA 은데/EC 몰입/NNP 도도/NNP 없/VA 고/EC 흥행/NNG 도/JX 패션왕/NNP 한테/JKB OO/SL 영화/NNG

SVM 은 support vector를 직접 확인할 수가 있습니다. 

linear SVM 의 support vector들을 확인해 보겠습니다. (4167, 4808) 은 4,808 차원 (단어 개수) 의 4,167 개의 support vectors 가 있다는 의미입니다. 이 숫자도 매우 큽니다. 이는 features 가 반복되지 않는 (각 리뷰별로 다양한 단어들이 이용되는) 데이터셋이기 때문입니다.

명사 추출기 때 살펴볼 (L, R) 구조의 데이터셋에 linear SVM 을 적용하면 support vectors 의 개수가 매우 작음을 볼 수 있습니다. 이는 이후에 명사 추출기 예시에서 살펴봅니다.

In [14]:
linear_SVM.support_vectors_.shape

(4167, 4808)

원래 데이터 X 에서의 각 support vectors 의 index 가 support_ 에 저장되어 있습니다.

In [15]:
linear_SVM.support_

array([   0,    1,    2, ..., 9989, 9994, 9996], dtype=int32)

SVC.dual\_coef\_에는 alpha가 학습되어 있습니다. 이 값이 negative이면 negative class 입니다. 

In [16]:
linear_SVM.dual_coef_

<1x4167 sparse matrix of type '<class 'numpy.float64'>'
	with 4167 stored elements in Compressed Sparse Row format>

In [17]:
import numpy as np

hist, bin_edges = np.histogram(
    linear_SVM.dual_coef_.data,
    bins=20
)

대부분의 support vectors 의 weight $\alpha$ 의 값이 -1 아니면 1 에 가깝습니다. 사실상 대부분의 리뷰들이 비슷한 중요도를 지닙니다.

In [18]:
for b, e, hist_i in zip(bin_edges, bin_edges[1:], hist):
    print('({:.3} ~ {:.3}) : {}'.format(b, e, hist_i))

(-1.0 ~ -0.9) : 1637
(-0.9 ~ -0.8) : 56
(-0.8 ~ -0.7) : 45
(-0.7 ~ -0.6) : 61
(-0.6 ~ -0.5) : 43
(-0.5 ~ -0.4) : 64
(-0.4 ~ -0.3) : 59
(-0.3 ~ -0.2) : 52
(-0.2 ~ -0.1) : 56
(-0.1 ~ 0.0) : 54
(0.0 ~ 0.1) : 33
(0.1 ~ 0.2) : 34
(0.2 ~ 0.3) : 33
(0.3 ~ 0.4) : 47
(0.4 ~ 0.5) : 45
(0.5 ~ 0.6) : 35
(0.6 ~ 0.7) : 31
(0.7 ~ 0.8) : 39
(0.8 ~ 0.9) : 48
(0.9 ~ 1.0) : 1695


## Naive Bayes

naive bayes 역시 학습 속도가 매우 빠릅니다. 

In [19]:
%%time

from sklearn.naive_bayes import BernoulliNB

bernoulli_naive_bayes = BernoulliNB()
bernoulli_naive_bayes.fit(x_norm, y)
test(bernoulli_naive_bayes, x, y, texts, test_samples)

[neg -> pos] 재미없/VA 다/EC 이상/NNG 10/SN 자/NNB
[neg -> neg] 다크/NNP 나이트/NNP 라/NNG 이즈/NNP 에/JKB 이/NP 은/JX 놀라/NNP ㄴ/JX 의/JKG 거품/NNG 이/JKS 확실히/MAG 드러나/VV ㄴ/ETM 영화/NNG 시각/NNG 효과/NNG 빼/VV 곤/EC 아무것/NNG 도/JX 보/VV ㄹ게/EC 없/VA 는/ETM 망/NNG 작/NNG
[neg -> pos] 시사회/NNP 보고/JKB 꿀/NNG 잠/NNG 불면증/NNP 치료/NNP 에/JKB
[neg -> neg] 알/VV 고/EC 보/VX 니/EC 지구/NNG 이/VCP 라는/ETM 설정/NNG 은/JX 혹성탈출/NNP 하/VV 고/EC 똑같/VA 지/EC 않/VX 어/EC 실망/NNP
[neg -> pos] 놀라/VV ㄴ/ETM 각본/NNP 은/JX 천재/NNP 이자/NNP 놀라/NNP ㄴ/JX 의/JKG 동생/NNG 조나단/NNP 놀라/VV ㄴ/ETM 영원/NNG 하/XSV ㄴ/ETM 거장/NNG 스티븐 스필버그/NNP 감독/NNG 제작/NNG 참여/NNG 한스/NNP 짐머/NNP 까지/JX OST/SL 하/XSV 아/EC 주/VX ㄴ다면/EC 길/VA ㄴ/ETM 말/NNG 필요/NNG 없이/MAG 일단/MAG 서/VV ㄴ/ETM 감상/NNP 천재/NNG 성/XSN 이/JKS 다분/XR 하/XSA ㄴ/ETM 놀라/VV ㄴ/ETM 형제/NNP 는/JX 지금껏/MAG 실망/NNG 시키/XSV ㄴ/ETM 적/XSN 이/JKS 없/VA 기/ETN 에/JKB 그만큼/MAG 사람/NNG 들/XSN 이/JKS 믿/VV 고/EC 보/VX 는/ETM 것/NNB 같/VA 은/ETM 소리/NNP 하네/NNP 핵/NNG 노/NNP 재/XPN 므/NNP
[neg -> neg] ㅅㅂ/NA 이런/MM 영화/NNG 제일/NNG 싫/VA 은데/EC 몰입/NNP 도도/NNP 없/VA 고/EC 흥행/NNG 도/JX 패션왕/NNP 한테/JKB OO/SL 영화/NNG

Naive Bayes 역시 binary classification 에서는 (1, n_terms) 의 coefficient 가 저장되어 있습니다. 숫자가 클수록 긍정 리뷰에 발생하는 단어라는 의미입니다.

일반적으로 확률의 곲을 이용하는 모델들은 계산의 편의성을 위하여 log 확률을 이용합니다. 확률의 누적곲에 log 를 취하면 덧샘이 되기 때문입니다. Naive Bayes 의 coefficient 의 값은 log 가 취해진 확률값입니다.

In [20]:
bernoulli_naive_bayes.coef_

array([[-6.57168296, -8.51759311, -4.32793837, ..., -5.95264375,
        -7.13129875, -5.33953928]])

하지만 Naive Bayes 는 단어 빈도수 정보까지 coefficient 에 저장합니다. Classsification 에 불필요한 단어가 많이 포함되어 있다면 이들에 의해 classification 의 성능이 저하될 수도 있습니다. 즉 Naive Bayes 를 잘 이용하기 위해서는 정보력이 적은 (frequent) 단어들은 제거해 주면 도움이 됩니다.

그리고 판별에 명확한 힌트가 되는 단어들로만 구성된 데이터셋이라면 coefficient 의 해석도 자연스럽습니다. 이 역시 명사 추출기에서 함께 살펴봅니다.

In [21]:
# Coefficients를 array로 만듭니다. 
coefficients = bernoulli_naive_bayes.coef_.reshape(-1)

# Coefficient를 (index, coef)로 만든 뒤, coefficient 의 크기로 내림차순 정렬합니다. 
sorted_coefficients = sorted(enumerate(coefficients), key=lambda x:-x[1])

for j, coef in sorted_coefficients[:10]:
    print('{} : {:.3}'.format(idx_to_vocab[j], coef))

관람객/NNG : -0.901
ㄴ/ETM : -1.07
보/VV : -1.13
영화/NNG : -1.16
이/VCP : -1.3
다/EC : -1.35
하/XSV : -1.44
는/ETM : -1.45
고/EC : -1.51
이/JKS : -1.52


In [22]:
for j, coef in sorted_coefficients[50:100]:
    print('{} : {:.3}'.format(idx_to_vocab[j], coef))

적/XSN : -2.85
들/XSN : -2.85
이/MM : -2.88
좋/VA : -2.93
지만/EC : -2.97
같/VA : -3.04
되/XSV : -3.04
다/JX : -3.05
내/NP : -3.06
되/VV : -3.08
생각/NNG : -3.09
음/ETN : -3.09
을/ETM : -3.1
싶/VX : -3.11
우주/NNG : -3.11
과/JC : -3.11
기/ETN : -3.12
필요/NNG : -3.12
만들/VV : -3.14
네요/EC : -3.15
감독/NNG : -3.16
않/VX : -3.17
진짜/MAG : -3.18
롭/XSA : -3.19
다시/MAG : -3.19
아서/EC : -3.2
잘/MAG : -3.2
보/VX : -3.21
재미있/VA : -3.22
인터스텔라/NNP : -3.22
감동/NNG : -3.23
하/VX : -3.23
더/MAG : -3.24
느끼/VV : -3.25
너무/MAG : -3.28
있/VX : -3.3
거/NNB : -3.34
이런/MM : -3.34
면/EC : -3.36
대단/XR : -3.37
나오/VV : -3.38
던/ETM : -3.42
점/NNB : -3.42
겠/EP : -3.42
까지/JX : -3.43
시간/NNG : -3.43
지/VX : -3.43
이해/NNG : -3.44
그/MM : -3.45
그냥/MAG : -3.47


## Decision Tree

In [23]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(
    max_depth=50,
    min_samples_split=2,
    min_samples_leaf=5,
    max_features=None,
    max_leaf_nodes=None,
    class_weight=None
)

decision_tree.fit(x_norm, y)
test(decision_tree, x, y, texts, test_samples)

[neg -> neg] 재미없/VA 다/EC 이상/NNG 10/SN 자/NNB
[neg -> neg] 다크/NNP 나이트/NNP 라/NNG 이즈/NNP 에/JKB 이/NP 은/JX 놀라/NNP ㄴ/JX 의/JKG 거품/NNG 이/JKS 확실히/MAG 드러나/VV ㄴ/ETM 영화/NNG 시각/NNG 효과/NNG 빼/VV 곤/EC 아무것/NNG 도/JX 보/VV ㄹ게/EC 없/VA 는/ETM 망/NNG 작/NNG
[neg -> neg] 시사회/NNP 보고/JKB 꿀/NNG 잠/NNG 불면증/NNP 치료/NNP 에/JKB
[neg -> neg] 알/VV 고/EC 보/VX 니/EC 지구/NNG 이/VCP 라는/ETM 설정/NNG 은/JX 혹성탈출/NNP 하/VV 고/EC 똑같/VA 지/EC 않/VX 어/EC 실망/NNP
[neg -> pos] 놀라/VV ㄴ/ETM 각본/NNP 은/JX 천재/NNP 이자/NNP 놀라/NNP ㄴ/JX 의/JKG 동생/NNG 조나단/NNP 놀라/VV ㄴ/ETM 영원/NNG 하/XSV ㄴ/ETM 거장/NNG 스티븐 스필버그/NNP 감독/NNG 제작/NNG 참여/NNG 한스/NNP 짐머/NNP 까지/JX OST/SL 하/XSV 아/EC 주/VX ㄴ다면/EC 길/VA ㄴ/ETM 말/NNG 필요/NNG 없이/MAG 일단/MAG 서/VV ㄴ/ETM 감상/NNP 천재/NNG 성/XSN 이/JKS 다분/XR 하/XSA ㄴ/ETM 놀라/VV ㄴ/ETM 형제/NNP 는/JX 지금껏/MAG 실망/NNG 시키/XSV ㄴ/ETM 적/XSN 이/JKS 없/VA 기/ETN 에/JKB 그만큼/MAG 사람/NNG 들/XSN 이/JKS 믿/VV 고/EC 보/VX 는/ETM 것/NNB 같/VA 은/ETM 소리/NNP 하네/NNP 핵/NNG 노/NNP 재/XPN 므/NNP
[neg -> neg] ㅅㅂ/NA 이런/MM 영화/NNG 제일/NNG 싫/VA 은데/EC 몰입/NNP 도도/NNP 없/VA 고/EC 흥행/NNG 도/JX 패션왕/NNP 한테/JKB OO/SL 영화/NNG

In [24]:
decision_tree = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=5,
    max_features=None,
    max_leaf_nodes=None,
    class_weight=None
)

decision_tree.fit(x_norm, y)
test(decision_tree, x, y, texts, test_samples)

[neg -> neg] 재미없/VA 다/EC 이상/NNG 10/SN 자/NNB
[neg -> neg] 다크/NNP 나이트/NNP 라/NNG 이즈/NNP 에/JKB 이/NP 은/JX 놀라/NNP ㄴ/JX 의/JKG 거품/NNG 이/JKS 확실히/MAG 드러나/VV ㄴ/ETM 영화/NNG 시각/NNG 효과/NNG 빼/VV 곤/EC 아무것/NNG 도/JX 보/VV ㄹ게/EC 없/VA 는/ETM 망/NNG 작/NNG
[neg -> neg] 시사회/NNP 보고/JKB 꿀/NNG 잠/NNG 불면증/NNP 치료/NNP 에/JKB
[neg -> neg] 알/VV 고/EC 보/VX 니/EC 지구/NNG 이/VCP 라는/ETM 설정/NNG 은/JX 혹성탈출/NNP 하/VV 고/EC 똑같/VA 지/EC 않/VX 어/EC 실망/NNP
[neg -> pos] 놀라/VV ㄴ/ETM 각본/NNP 은/JX 천재/NNP 이자/NNP 놀라/NNP ㄴ/JX 의/JKG 동생/NNG 조나단/NNP 놀라/VV ㄴ/ETM 영원/NNG 하/XSV ㄴ/ETM 거장/NNG 스티븐 스필버그/NNP 감독/NNG 제작/NNG 참여/NNG 한스/NNP 짐머/NNP 까지/JX OST/SL 하/XSV 아/EC 주/VX ㄴ다면/EC 길/VA ㄴ/ETM 말/NNG 필요/NNG 없이/MAG 일단/MAG 서/VV ㄴ/ETM 감상/NNP 천재/NNG 성/XSN 이/JKS 다분/XR 하/XSA ㄴ/ETM 놀라/VV ㄴ/ETM 형제/NNP 는/JX 지금껏/MAG 실망/NNG 시키/XSV ㄴ/ETM 적/XSN 이/JKS 없/VA 기/ETN 에/JKB 그만큼/MAG 사람/NNG 들/XSN 이/JKS 믿/VV 고/EC 보/VX 는/ETM 것/NNB 같/VA 은/ETM 소리/NNP 하네/NNP 핵/NNG 노/NNP 재/XPN 므/NNP
[neg -> neg] ㅅㅂ/NA 이런/MM 영화/NNG 제일/NNG 싫/VA 은데/EC 몰입/NNP 도도/NNP 없/VA 고/EC 흥행/NNG 도/JX 패션왕/NNP 한테/JKB OO/SL 영화/NNG

In [25]:
import numpy as np
np.where(decision_tree.feature_importances_ > 0)[0]

array([   2,  161,  162,  178,  179,  184,  206,  257,  336,  480,  494,
        559,  585,  842,  877,  882,  931,  985, 1006, 1018, 1031, 1032,
       1074, 1111, 1152, 1255, 1299, 1406, 1557, 1591, 1662, 2078, 2114,
       2512, 2601, 2716, 2734, 2741, 2808, 2851, 2983, 2996, 3098, 3359,
       3368, 3375, 3408, 3435, 3551, 3697, 3706, 3876, 3947, 3973, 3998,
       4000, 4008, 4176, 4213, 4456, 4511, 4517, 4518, 4558])

In [26]:
decision_tree.feature_importances_.shape

(4808,)

몇 개의 단어만을 이용하여 분류를 하다보니 분류가 제대로 이뤄지지 않기도 합니다. 게다가 분류에 유용한 features 가 아닌 경우도 많습니다. 또한 `크리스토퍼 놀란` 때문에 `놀라/VV`, `놀라/NNP` 와 같은 잘못된 features 들도 생성되었음을 볼 수 있습니다.

In [27]:
[(idx_to_vocab[idx], importancy) for idx, importancy in 
 sorted(
     filter(lambda x:x[1]>0,
            enumerate(decision_tree.feature_importances_)),
     key=lambda x:-x[1]
 )
]

[('관람객/NNG', 0.4510845086787639),
 ('놀라/VV', 0.10623425004513344),
 ('최고/NNG', 0.08842707720332754),
 ('번/NNB', 0.03825325790437422),
 ('경이/NNG', 0.03468372593892105),
 ('필요/NNG', 0.029884436449725816),
 ('수/NNB', 0.02891829972680556),
 ('놀라/NNP', 0.027174462181816735),
 ('아깝/VA', 0.023680759018578437),
 ('인터스텔라/NNP', 0.02090671738172154),
 ('재미없/VA', 0.015089595722997775),
 ('들/XSN', 0.01022028341450091),
 ('지루/XR', 0.009356473444929085),
 ('평점/NNG', 0.008016104496944376),
 ('노/NNG', 0.006661564598202253),
 ('지/EC', 0.006367177178232638),
 ('1/SN', 0.005127239183416747),
 ('졸/VV', 0.004864729250959874),
 ('ㄴ/ETM', 0.004777572451144816),
 ('을/JKO', 0.00454902543036219),
 ('쓰레기/NNP', 0.004385307060655613),
 ('ㄹ/ETM', 0.004208895713729889),
 ('나/NP', 0.0040954016849899496),
 ('ㅁ/ETN', 0.0036113313444510104),
 ('없/VA', 0.0029579094762203235),
 ('음/ETN', 0.002657335600262841),
 ('난해/XR', 0.0026438404474042445),
 ('지루/NNG', 0.0026313753816456517),
 ('ㄴ지/EC', 0.0026216086307932697),
 ('나/EC'